In [24]:
#載入資料
import csv
import numpy as np

# 開啟 CSV 檔案
with open('106年樞紐分析表(有交叉路名及其他地點).csv', newline='',encoding="big5") as accident:
    accident = csv.reader(accident)
    accident = [row for row in accident]



In [25]:
for i in range(len(accident[0])):
    print(i,accident[0][i], end = " ")
print("")

effective_item = [6,7,12,13,19,20,21,23,24,25,27,28,29,30,31,32,40]
for i in effective_item:
    print(i,accident[0][i])

for i in accident:
    #print(i[7],end = " ")
    try:
        if "不明原因肇事" in i[13]:
            del i[13] 
        if i[7] == "-1":
            del i[7]
        else:
            # age 轉換成年齡範圍
            i[7] = str(int(i[7])//10 * 10)
            #print(i[7])

            
    except:
        pass
    

0  發生年 1  發生月 2  發生日 3  處理單位(分局別) 4  處理單位(派出所) 5  鄉鎮市 6  道路型態 7  年齡 8  屬性 9  死亡人數 10  受傷人數 11  當事人別 12  當事者區分 13  肇事原因 14  路段 15  巷 16  弄 17  門牌號 18  交叉路名 19  事故類別 20  發生時間 21  事故類型及型態 22  其他地點 23  天候 24  光線 25  道路類別 26  速限 27  事故位置 28  路面狀況-路面鋪裝 29  路面狀況-路面狀態 30  路面狀況-路面缺陷 31  道路障礙-障礙物視距 32  道路障礙-視距 33  號誌-號誌種類 34  號誌-號誌動作 35  車道劃分設施-分道設施-快車道或一般車道間 36  車道劃分設施-分道設施-快慢車道間 37  車道劃分設施-分道設施-路面邊線 38  車道劃分設施-分向設施 39  保護裝備 40  當事者行動狀態 41  駕駛資格情形 42  駕駛執照種類 43  飲酒情形 44  交通事故逃逸 45  受傷程度 46  主要傷處 47  行動電話、電腦或其他相類功能裝置 48  肇事逃逸 49  公路 50  公里 51  公尺 52  方向 53  車道 54  件數 
6  道路型態
7  年齡
12  當事者區分
13  肇事原因
19  事故類別
20  發生時間
21  事故類型及型態
23  天候
24  光線
25  道路類別
27  事故位置
28  路面狀況-路面鋪裝
29  路面狀況-路面狀態
30  路面狀況-路面缺陷
31  道路障礙-障礙物視距
32  道路障礙-視距
40  當事者行動狀態


In [26]:
for i in effective_item:
    print(i,accident[1][i])

6 14-直路
7 30
12 B03-自用
13 五股區中興路2段
19 9
20 車與車-對撞
21 
23 日間自然光線
24 市區道路
25 50
27 柏油
28 乾燥
29 無缺陷
30 無障礙物
31 良好
32 無號誌
40 有適當之駕照


In [ ]:
#https://zhuanlan.zhihu.com/p/39918644
#別人寫的算法，但是沒辦法直接用求優化
# !/usr/bin/env python
# -*-  coding:utf-8  -*-
# version: Python 3.6.4 on win64
# author:  Suranyi    time:  7/17
# title:  Apriori算法

import itertools
import pandas as pd

class Apriori(object):
    def __init__(self, itemSets, minSupport=0.5, minConf=0.7):
        self.itemSets = itemSets
        self.minSupport = minSupport
        self.minConf = minConf
        self.supportDates = dict()
        self.support_select = dict()
        self.confidence_select = dict()
        self.__Initialize()

    def __item(self):
        '''获取项目元素列表'''
        self.item = []
        for itemSet in self.itemSets:
            for item in itemSet:
                if item not in self.item:
                    self.item.append(item)
        self.item.sort()

    def __count_dict(self):
        '''为每个可能的候选集计数'''
        for itemSet in self.itemSets:
            for i in range(1, len(itemSet)+1):
                for factor in list(itertools.combinations(itemSet, i)):
                    self.supportDates.setdefault(','.join(factor), 0)
                    self.supportDates[','.join(factor)] += 1 / len(self.itemSets)

    def __support_select_fun(self):
        '''选择所有符合最小支持度要求的元素作为频繁项集'''
        for k, v in self.supportDates.items():
            if v >= self.minSupport:
                self.support_select[k] = v

    def __confidence_select_fun(self):
        '''选择所有符合最小自信度要求的元素作为关联规则'''
        for k, v in self.support_select.items():
            for i in range(1, len(self.item) - len(k.split(',')) + 1):
                for factor in list(itertools.combinations(set(self.item) - set(k.split(',')), i)):
                    if self.support_select.get(','.join(sorted(k.split(',') + list(factor)))):
                        Supp = self.support_select[','.join(sorted(k.split(',') + list(factor)))]
                        Conf = Supp / self.support_select[k]
                        if Conf >= self.minConf:
                            self.confidence_select[k + '-->' + ','.join(factor)] = {'Support': Supp, 'Confidence': Conf}

    def __visualization(self):
        '''可视化输出'''
        print(self.confidence_select)
        #print(pd.DataFrame(self.confidence_select, index=['Support', 'Confidence']).T.sort_values(by=['Support', 'Confidence'], ascending=False))

    def __Initialize(self):
        '''初始化函数，执行一次'''
        self.__item()
        self.__count_dict()
        self.__support_select_fun()
        self.__confidence_select_fun()
        self.__visualization()

    def update(self, minSupport, minConf):
        '''用于更新数据'''
        self.minSupport = minSupport
        self.minConf = minConf
        self.support_select = dict()
        self.confidence_select = dict()
        self.__support_select_fun()
        self.__confidence_select_fun()
        self.__visualization()

def str2itemsets(strings, split=','):
    '''将字符串列表转化为对应的集合'''
    itemsets = []
    for string in strings:
        itemsets.append(sorted(string.split(split)))
    return itemsets

if __name__ == '__main__':
    #data = ['紫菜蛋花汤,糖醋排骨,爆炒花甲', '红烧里脊,酸辣鱼', '红烧里脊,糖醋排骨', '紫菜蛋花汤,红烧里脊,糖醋排骨,酸辣鱼', '紫菜蛋花汤,红烧里脊', '红烧里脊,糖醋排骨', '紫菜蛋花汤,红烧里脊', '紫菜蛋花汤,红烧里脊,糖醋排骨,爆炒花甲', '紫菜蛋花汤,红烧里脊,糖醋排骨', '紫菜蛋花汤,糖醋排骨,爆炒花甲']

    #itemSets = str2itemsets(data)  # 转化为标准格式
    itemSets = accident[:10]
    a = Apriori(itemSets, 0.2, 0.6)
    a.update(0.4, 0.7) # 更新修改这里